In [1]:
import os
import time
import argparse
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from dataset import Shuttlecock_Trajectory_Dataset
from keypoint_dataset import KeypointDataset
from test import eval_tracknet, eval_inpaintnet
from utils.general import ResumeArgumentParser, get_model, to_img_format
from utils.metric import WBCELoss
from utils.visualize import plot_heatmap_pred_sample, plot_traj_pred_sample, write_to_tb
import matplotlib.pyplot as plt

In [2]:
cuda_device_number = 3
device = torch.device(f"cuda:{cuda_device_number}")
torch.cuda.set_device(cuda_device_number)

In [3]:
tracknet_best_ckpt = torch.load(os.path.join("ckpts/TrackNet_best.pt"), map_location=device)
inpaintnet_best_ckpt = torch.load(os.path.join("ckpts/InpaintNet_best.pt"), map_location=device)
# keypointnet_best_ckpt = torch.load(os.path.join("finetune/KeypointNet_best.pt"), map_location=device)


In [5]:
track_net = get_model("TrackNet", 8, "concat").cuda()
inpaint_net = get_model("InpaintNet").cuda()
keypoint_net = get_model("KeypointNet", 9).cuda()

In [6]:
track_net.load_state_dict(tracknet_best_ckpt['model'])
inpaint_net.load_state_dict(inpaintnet_best_ckpt['model'])
# keypoint_net.load_state_dict(keypointnet_best_ckpt['model'])

<All keys matched successfully>

In [11]:
train_dataset = KeypointDataset(split='train', seq_len=9, sliding_step=1, frame_alpha=-1, debug=False)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=1, drop_last=True, pin_memory=True)

Generate input file...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:00<00:00, 141.47it/s]


In [21]:
counter = 0
for item in train_loader:
    break

_, frames, heatmaps, coor, _, keypoint = item
coor = coor.cuda()

In [18]:
inp = coor.reshape(len(coor), -1).float()
y_pred = keypoint_net(inp)

In [23]:
y = keypoint
softmax = nn.Softmax(1)
print(y[:, 9//2])
print(softmax(y_pred))

tensor([0., 0.], dtype=torch.float64)
tensor([[0.1408, 0.1934, 0.1593, 0.3560, 0.1505],
        [0.1408, 0.1934, 0.1593, 0.3560, 0.1505]], device='cuda:3',
       grad_fn=<SoftmaxBackward0>)


In [10]:
x1 = inp[0]
x1 = keypoint_net.fc_in(x1)
# x1 = keypoint_net.relu(x1)
# x1 = keypoint_net.dropout(x1)
# for block in keypoint_net.residual_blocks:
#     x1 = block(x1)
# x1 = keypoint_net.fc_out(x1)


In [11]:
x2 = inp[1]
x2 = keypoint_net.fc_in(x2)
# x2 = keypoint_net.relu(x2)
# x2 = keypoint_net.dropout(x2)
# for block in keypoint_net.residual_blocks:
#     x2 = block(x2)
# x2 = keypoint_net.fc_out(x2)

In [13]:
x1-x2

tensor([ 0.2093, -0.0776,  0.2255,  0.0836, -0.9063, -0.2458,  0.2613, -0.0298,
        -0.3951,  0.0196,  0.5369,  0.0204,  0.0352,  0.0079,  0.1435,  0.2769,
         0.1116, -0.0822, -0.0504,  0.0975, -0.1286,  0.4943, -0.4723,  0.1664,
         0.1295, -1.0265,  0.0082,  0.0608,  0.1541, -0.2778, -0.1698, -0.5654,
         0.1531,  0.1706,  0.2792, -0.2909,  0.0418, -0.1711, -0.0865, -0.0078,
         0.0946, -0.0709,  0.0191, -0.4431,  0.2542,  0.1905, -0.3456, -0.2261,
         0.2651, -0.0524,  0.1583, -0.3851, -0.1265,  0.2317,  0.1912,  0.0403,
        -0.0075, -0.1402,  0.2496, -0.1792,  0.2213, -0.2941,  0.1435, -0.1234],
       device='cuda:3', grad_fn=<SubBackward0>)